(complementary:time)=
# Time dependant problems
In this unit we want to turn to instationary problems. And we will start with a very basic setup: implicit Euler time stepping for the convection diffusion equation. After the main part of this tutorial we have supplementary material to extend the case in several regards.



We are solving the unsteady heat equation 

$$\text{find } u:[0,T] \to H_{0,D}^1 \quad \int_{\Omega} \partial_t u v + \int_{\Omega} \nabla u \nabla v + b \cdot \nabla u v = \int f v  \quad \forall v \in H_{0,D}^1, \quad u(t=0) = u_0$$

with a suitable advective field $b$ (the wind).

In [ ]:
#imports
from ngsolve import *
#from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw

* Geometry: $(-1,1)^2$
* Dirichlet boundaries everywhere
* Mesh

In [ ]:
from netgen.occ import *
from netgen.webgui import Draw as DrawGeo
shape = Rectangle(2,2).Face().Move((-1,-1,0))
shape.edges.Min(X).name="left"
shape.edges.Max(X).name="right"
shape.edges.Min(Y).name="bottom"
shape.edges.Max(Y).name="top"
mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=0.25))

In [ ]:
fes = H1(mesh, order=3, dirichlet="bottom|right|left|top")
u,v = fes.TnT()
time = 0.0
dt = 0.001

We define the field $b$ (the wind) as 

$$b(x,y) := (2y(1-x^2),-2x(1-y^2)).$$

In [ ]:
b = CoefficientFunction((2*y*(1-x*x),-2*x*(1-y*y)))
Draw(b,mesh,"wind", vectors={"grid_size": 32}, order=3)

* bilinear forms for 
 * convection-diffusion stiffness and 
 * mass matrix separately.
* non-symmetric memory layout for the mass matrix so that a and m have the same sparsity pattern.

In [ ]:
a = BilinearForm(fes, symmetric=False)
a += 0.01*grad(u)*grad(v)*dx + b*grad(u)*v*dx
a.Assemble()

m = BilinearForm(fes, symmetric=False)
m += u*v*dx
m.Assemble()

## Implicit Euler method
$$
  \underbrace{(M + \Delta t A)}_{M^\ast} u^{n+1} = M u^n + \Delta tf^{n+1}
$$

or in an incremental form:

$$
  M^\ast (u^{n+1} - u^n) = - \Delta t A u^n + \Delta tf^{n+1}.
$$

* Incremental form: $u^{n+1} - u^n$ has homogeneous boundary conditions (unless boundary conditions are time-dependent).
* For the time stepping method: set up linear combinations of matrices.
* (Only) if the sparsity pattern of the matrices agree we can copy the pattern and sum up the entries.

First, we create a matrix of same format as m.mat and compare number of non-zero entries and sparsity pattern:

In [ ]:
mstar = m.mat.CreateMatrix()
print(f"m.mat.nze = {m.mat.nze}, a.mat.nze={a.mat.nze}, mstar.nze={mstar.nze}")

Using the vector we can build the linear combination of the a and the m matrix:

In [ ]:
mstar.AsVector().data = m.mat.AsVector() + dt * a.mat.AsVector()

# We cannot do this:
# mstar.data = m.mat + dt * a.mat
# unsupported operand type(s) for *: 'NoneType' and 'ngsolve.la.DynamicVectorExpression'

# corresponds to M* = M + dt * A
invmstar = mstar.Inverse(freedofs=fes.FreeDofs())

We set the r.h.s. $f = exp(-6 ((x+\frac12)^2+y^2)) - exp(-6 ((x-\frac12)^2+y^2))$

In [ ]:
f = LinearForm(fes)
gaussp = exp(-6*((x+0.5)*(x+0.5)+y*y))-exp(-6*((x-0.5)*(x-0.5)+y*y))
Draw(gaussp,mesh,"f", deformation=True)
f += gaussp*v*dx
f.Assemble()

and the initial data: $u_0 = (1-y^2)x$

In [ ]:
gfu = GridFunction(fes)
gfu.Set((1-y*y)*x) # note that boundary conditions remain
scene = Draw(gfu,mesh,"u")

we define a simple time loop including some visualization sampling:

In [ ]:
def TimeStepping(invmstar, initial_cond = None, t0 = 0, tend = 2, 
                 nsamples = 10):
    if initial_cond:
        gfu.Set(initial_cond)
    cnt = 0; time = t0
    sample_int = int(floor(tend / dt / nsamples)+1)
    
    gfut = GridFunction(gfu.space,multidim=0)
    gfut.AddMultiDimComponent(gfu.vec)

    while time < tend - 0.5 * dt:
        res = dt * f.vec - dt * a.mat * gfu.vec
        gfu.vec.data += invmstar * res
        print("\r",time,end="")
        scene.Redraw()
        if cnt % sample_int == 0:
            gfut.AddMultiDimComponent(gfu.vec)
        cnt += 1; time = cnt * dt
    return gfut

In [ ]:
%%time
gfut = TimeStepping(invmstar, (1-y*y)*x)

 1.999CPU times: user 7.05 s, sys: 630 ms, total: 7.68 s
Wall time: 6.57 s

In [ ]:
Draw(gfut, mesh, interpolate_multidim=True, animate=True)

## Using iterative solvers

* For a factorization of $M^\ast$ (and ${M^\ast}^{-1}$) we required a sparse matrix $M^\ast$ 
* To store $M^\ast$ **as a sparse matrix** requires new storage (and same memory layout of $A$ and $M$)
* For iterative solvers we only require the matrix (and preconditioner) applications and it **suffices to have $M^\ast$ available as a linear operator**
* `mstar_alt = m.mat + dt * a.mat` has no storage but defines the operator action: sum of two matrix-vector multiplications

iterative solver version (with Jacobi preconditining):

** does not perform well for this problem, just for explanation **



In [ ]:
mstar_alt = m.mat + dt * a.mat

precond = preconditioners.Local
preflags ={"GS":False}

prea = precond(a, **preflags)
prem = precond(m, **preflags)


premstar_alt = prem.mat + dt * prea.mat
print(premstar_alt)

Now, replace the action of the inverse Matrix from the previous script with a solution of a conjugate gradient method:

In [ ]:
from ngsolve.krylovspace import CGSolver
invmstar_alt = CGSolver(mstar_alt, premstar_alt, tol=1e-8, plotrates=False, maxiter=200)

Now, we can repeat the time stepping. As in 2D direct solvers are quite efficient, this simple preconditioned solver can hardly compete. We hence only do a few time steps:

In [ ]:
%%time
gfut_a1 = TimeStepping(invmstar_alt, (1-y*y)*x )

In [ ]:
Draw(gfut_a1, mesh, interpolate_multidim=True, animate=True);

## time-dependent r.h.s. data 

Next: time-dependent r.h.s. data $f=f(t)$:

* Use `Parameter` t representing the time. 
* A `Parameter` is a constant `CoefficientFunction` the value of which can be changed with the `Set`-function.

In [ ]:
t = Parameter(0.0)

An example of a time-dependent coefficient that we want to use as r.h.s. in the following is

In [ ]:
omega=pi
gausspt = exp(-10*((x-0.5)**2 + (y-0.5)**2) )*(sin(omega*t))
gff = GridFunction(L2(mesh,order=gfu.space.globalorder+1))
gfft = GridFunction(gff.space,multidim=0)
time = 0.0
for i in range(7):
    t.Set(3*i/6)
    gff.Set(gausspt)
    gfft.AddMultiDimComponent(gff.vec)
Draw(gfft, mesh, interpolate_multidim=True, animate=True, min=-1, max=1, autoscale=False);

Accordingly, we define a different linear form which then has to be assembled in every time step.

In [ ]:
ft = LinearForm(fes)
ft += gausspt*v*dx

In [ ]:
def TimeStepping_app2(invmstar, initial_cond = None, t0 = 0, tend = 2, 
                      nsamples = 20):
    if initial_cond:
        gfu.Set(initial_cond)
    cnt = 0; time = t0
    sample_int = int(floor(tend / dt / nsamples)+1)
    gfut = GridFunction(gfu.space,multidim=0)
    gfut.AddMultiDimComponent(gfu.vec)
    while time < tend - 0.5 * dt:
        t.Set(time)
        ft.Assemble()
        res = dt * ft.vec - dt * a.mat * gfu.vec
        gfu.vec.data += invmstar * res
        print("\r",time,end="")
        if cnt % sample_int == 0:
            gfut.AddMultiDimComponent(gfu.vec)
        cnt += 1; time = cnt * dt
    return gfut

In [ ]:
%%time
gfut_a2 = TimeStepping_app2(invmstar, initial_cond=CF(0),tend=5)
Draw(gfut_a2, mesh, interpolate_multidim=True, animate=True, 
     min=-0.25,max=0.25,autoscale=False);

## Time dependent boundary conditions


* $u|_{\partial \Omega} = u_D(t)$, $f=0$
* implicit Euler time stepping method, non-incremental form:

  $$
    M^\ast u^{n+1} = (M + \Delta t A) u^{n+1} = M u^n
  $$  
  
* Homogenize w.r.t. to boundary conditions, i.e. we split 

  $$ u^{n+1} = u^{n+1}_0 + u^{n+1}_D $$ 
  
  where $u^{n+1}_D$ is a (discrete) function with correct boundary condition:  
  
$$
  {M^\ast} u^{n+1}_0 = M u^n - {M^\ast} u^{n+1}_D
$$


In [ ]:
uD = CoefficientFunction( (cos(5*(x+t))*y ))
time = 0.0
t.Set(0.0)
gfu.Set(uD,BND)
Draw(gfu,mesh,"uD");

In [ ]:
def TimeStepping_app3(invmstar, initial_cond = None, t0 = 0, tend = 2, 
                      nsamples = 10):
    if initial_cond:
        gfu.Set(initial_cond)
    cnt = 0; time = t0
    sample_int = int(floor(tend / dt / nsamples)+1)
    gfuD = GridFunction(gfu.space)
    gfut = GridFunction(gfu.space,multidim=0)
    gfut.AddMultiDimComponent(gfu.vec)
    while time < tend - 0.5 * dt:
        t.Set(time)
        gfuD.Set(uD,BND)
        res = m.mat * gfu.vec - mstar *gfuD.vec
        gfu.vec.data = gfuD.vec + invmstar * res
        print("\r",time,end="")
        if cnt % sample_int == 0:
            gfut.AddMultiDimComponent(gfu.vec)
        cnt += 1; time = cnt * dt
    return gfut

In [ ]:
%%time
gfut_a3 = TimeStepping_app3(invmstar, initial_cond=CF(gaussp),tend=2)


In [ ]:

Draw(gfut_a3, mesh, interpolate_multidim=True, animate=True, 
     #settings = {"subdivision" : 10}, 
     deformation = False, min = 0, max = 1, autoscale = True);